In [40]:
"""
Save this as: stable_table_detector.ipynb

STABLE BPHS TABLE DETECTION SYSTEM (NO PADDLEOCR)

✓ Dual-method table detection:
  1. DETR Table Transformer (two-pass: 0.6, 0.4)
  2. OpenCV contour detection
✓ Merge & deduplicate overlapping boxes
✓ Google Vision OCR for multilingual text (Hindi, Sanskrit, English)
✓ Extremely stable - no crashes
✓ Save formatted text file + JSON with verification images

Installation:
!pip install opencv-python torch transformers pdf2image pillow google-cloud-vision

Env:
export GOOGLE_APPLICATION_CREDENTIALS="vision_key.json"
"""


'\nSave this as: stable_table_detector.ipynb\n\nSTABLE BPHS TABLE DETECTION SYSTEM (NO PADDLEOCR)\n\n✓ Dual-method table detection:\n  1. DETR Table Transformer (two-pass: 0.6, 0.4)\n  2. OpenCV contour detection\n✓ Merge & deduplicate overlapping boxes\n✓ Google Vision OCR for multilingual text (Hindi, Sanskrit, English)\n✓ Extremely stable - no crashes\n✓ Save formatted text file + JSON with verification images\n\nInstallation:\n!pip install opencv-python torch transformers pdf2image pillow google-cloud-vision\n\nEnv:\nexport GOOGLE_APPLICATION_CREDENTIALS="vision_key.json"\n'

In [41]:
import sys
import cv2
import numpy as np
import torch

print("Python:", sys.executable)
print("cv2:", cv2.__version__)
print("numpy:", np.__version__)
print("torch:", torch.__version__)


Python: c:\Users\saini\OneDrive\Desktop\IMG_Global_infotech\ocr_env\Scripts\python.exe
cv2: 4.6.0
numpy: 1.26.4
torch: 2.2.1+cpu


In [42]:

# Cell 1: Imports and Setup
import json
import cv2
import torch
import numpy as np
from pathlib import Path
from pdf2image import convert_from_path
from PIL import Image
from google.cloud import vision
from transformers import DetrImageProcessor, TableTransformerForObjectDetection
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [43]:
# Cell 2: Configuration
# ---------------- CONFIG ----------------

PDF_PATH = "bphs_table_samples 2.pdf"
OUTPUT_DIR = "final_output"
DPI = 300
TABLE_THRESHOLD_PASS1 = 0.6
TABLE_THRESHOLD_PASS2 = 0.4
IOU_THRESHOLD = 0.3  # Lower for better merging

# Google Vision credentials path
VISION_CREDENTIALS = r"C:\Users\saini\OneDrive\Desktop\IMG_Global_infotech\google_credentials 1.json"

# Create output directory (with parents if needed)
import os
output_path = Path(OUTPUT_DIR)
output_path.mkdir(parents=True, exist_ok=True)
print(f"✅ Output directory created: {output_path.absolute()}")

# Verify setup
print("\n" + "="*70)
print("🔍 SETUP VERIFICATION")
print("="*70)

# Check PDF exists
if os.path.exists(PDF_PATH):
    print(f"✅ PDF file found: {PDF_PATH}")
else:
    print(f"❌ PDF file NOT found: {PDF_PATH}")
    print(f"   Current directory: {os.getcwd()}")

# Check credentials exist
if os.path.exists(VISION_CREDENTIALS):
    print(f"✅ Google credentials found")
else:
    print(f"❌ Google credentials NOT found")

print("="*70)

✅ Output directory created: c:\Users\saini\OneDrive\Desktop\IMG_Global_infotech\final_output

🔍 SETUP VERIFICATION
✅ PDF file found: bphs_table_samples 2.pdf
✅ Google credentials found


In [44]:
# Cell 3: Initialize OCR Systems
# ---------------- INITIALIZE CLIENTS ----------------

print("Initializing OCR system...")

# Google Vision - Primary and most reliable OCR
vision_client = vision.ImageAnnotatorClient.from_service_account_file(VISION_CREDENTIALS)
print("✅ Google Vision initialized (supports Hindi, Sanskrit, English)")


Initializing OCR system...
✅ Google Vision initialized (supports Hindi, Sanskrit, English)


In [45]:
# Cell 4: Load Table Transformer Model
# ---------------- TABLE TRANSFORMER MODEL ----------------

print("\nLoading DETR Table Transformer...")
processor = DetrImageProcessor.from_pretrained("microsoft/table-transformer-detection")
table_model = TableTransformerForObjectDetection.from_pretrained(
    "microsoft/table-transformer-detection"
)
print("✅ DETR model loaded")


Loading DETR Table Transformer...


Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✅ DETR model loaded


In [46]:
# Cell 5: Helper Functions
# ---------------- HELPER FUNCTIONS ----------------

def calculate_iou(box1, box2):
    """Calculate Intersection over Union of two boxes."""
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    
    if x2_i < x1_i or y2_i < y1_i:
        return 0.0
    
    intersection = (x2_i - x1_i) * (y2_i - y1_i)
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0.0

def merge_boxes(boxes, iou_threshold=0.3):
    """Merge overlapping boxes from all detection methods."""
    if len(boxes) == 0:
        return []
    
    merged = []
    used = [False] * len(boxes)
    
    for i in range(len(boxes)):
        if used[i]:
            continue
            
        current_box = list(boxes[i])
        used[i] = True
        
        for j in range(i + 1, len(boxes)):
            if used[j]:
                continue
                
            iou = calculate_iou(current_box, boxes[j])
            if iou > iou_threshold:
                current_box = [
                    min(current_box[0], boxes[j][0]),
                    min(current_box[1], boxes[j][1]),
                    max(current_box[2], boxes[j][2]),
                    max(current_box[3], boxes[j][3])
                ]
                used[j] = True
        
        merged.append(current_box)
    
    return merged

def draw_dashed_rect(img, pt1, pt2, color=(0, 255, 0), thickness=3, dash=10):
    """Draw dashed rectangle."""
    x1, y1 = pt1
    x2, y2 = pt2

    for x in range(x1, x2, dash * 2):
        cv2.line(img, (x, y1), (min(x + dash, x2), y1), color, thickness)
        cv2.line(img, (x, y2), (min(x + dash, x2), y2), color, thickness)

    for y in range(y1, y2, dash * 2):
        cv2.line(img, (x1, y), (x1, min(y + dash, y2)), color, thickness)
        cv2.line(img, (x2, y), (x2, min(y + dash, y2)), color, thickness)

print("✅ Helper functions defined")

✅ Helper functions defined


In [47]:
# Cell 6: PDF to Images Conversion
# ---------------- PDF → IMAGES ----------------

def pdf_to_images(pdf_path):
    """Convert PDF to high-resolution images."""
    print(f"Converting PDF to images (DPI: {DPI})...")
    images = convert_from_path(pdf_path, dpi=DPI)
    paths = []
    for i, img in enumerate(images):
        p = Path(OUTPUT_DIR) / f"page_{i+1}.png"
        img.save(p, "PNG")
        paths.append(str(p))
    print(f"✅ Converted {len(paths)} pages")
    return paths

print("✅ PDF conversion function defined")

✅ PDF conversion function defined


In [48]:
# Cell 7: Table Detection Methods
# ---------------- TABLE DETECTION METHODS ----------------

def detect_tables_detr(image_path, threshold):
    """Method 1: DETR Table Transformer detection."""
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = table_model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(
        outputs, threshold=threshold, target_sizes=target_sizes
    )[0]

    return results["boxes"].tolist()

def detect_tables_opencv(image_path):
    """Method 2: OpenCV contour-based table detection."""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Thresholding
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    
    # Detect horizontal and vertical lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
    
    horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel)
    vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel)
    
    # Combine lines
    table_mask = cv2.add(horizontal, vertical)
    
    # Find contours
    contours, _ = cv2.findContours(table_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    boxes = []
    min_area = (img.shape[0] * img.shape[1]) * 0.01  # Minimum 1% of page
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        
        if area > min_area and w > 100 and h > 100:
            boxes.append([x, y, x + w, y + h])
    
    return boxes

def detect_tables_all_methods(image_path):
    """Combine DETR and OpenCV detection methods."""
    all_boxes = []
    
    # Method 1: DETR (two-pass)
    print("  🔍 DETR Pass 1 (threshold 0.6)...")
    boxes_detr_1 = detect_tables_detr(image_path, TABLE_THRESHOLD_PASS1)
    print(f"    Found: {len(boxes_detr_1)} tables")
    all_boxes.extend(boxes_detr_1)
    
    print("  🔍 DETR Pass 2 (threshold 0.4)...")
    boxes_detr_2 = detect_tables_detr(image_path, TABLE_THRESHOLD_PASS2)
    print(f"    Found: {len(boxes_detr_2)} tables")
    all_boxes.extend(boxes_detr_2)
    
    # Method 2: OpenCV
    print("  🔍 OpenCV contour detection...")
    boxes_opencv = detect_tables_opencv(image_path)
    print(f"    Found: {len(boxes_opencv)} tables")
    all_boxes.extend(boxes_opencv)
    
    # Merge all detections
    print(f"  📦 Total boxes before merging: {len(all_boxes)}")
    merged = merge_boxes(all_boxes, IOU_THRESHOLD)
    print(f"  ✅ Final unique tables: {len(merged)}")
    
    return merged

print("✅ Table detection methods defined")


✅ Table detection methods defined


In [49]:
# Cell 8: Text Extraction Functions (COMPLETE FIX)
# ---------------- TEXT EXTRACTION ----------------

def extract_text_with_structure(image_path):
    """
    Extract text with structure using Google Vision API.
    Returns blocks with bounding boxes and full text.
    """
    # Read image file
    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    
    # Create Vision API image object
    image = vision.Image(content=content)
    
    # Perform document text detection
    response = vision_client.document_text_detection(image=image)
    
    if response.error.message:
        raise Exception(f"Google Vision API error: {response.error.message}")
    
    # Extract full text
    full_text = response.full_text_annotation.text if response.full_text_annotation else ""
    
    # Extract structured blocks with bounding boxes
    blocks = []
    if response.full_text_annotation:
        for page in response.full_text_annotation.pages:
            for block in page.blocks:
                # Get bounding box vertices
                vertices = []
                for vertex in block.bounding_box.vertices:
                    vertices.append({"x": vertex.x, "y": vertex.y})
                
                # Extract text from block
                block_text = ""
                for paragraph in block.paragraphs:
                    for word in paragraph.words:
                        word_text = "".join([symbol.text for symbol in word.symbols])
                        block_text += word_text + " "
                
                blocks.append({
                    "text": block_text.strip(),
                    "bounding_box": vertices
                })
    
    return blocks, full_text


def extract_text_google_vision(image_path):
    """Extract text using Google Vision OCR."""
    blocks, full_text = extract_text_with_structure(image_path)
    return blocks, full_text


def extract_table_text(image_path, table_box):
    """Extract text from a specific table region."""
    img = cv2.imread(image_path)
    x1, y1, x2, y2 = map(int, table_box)
    
    # Add padding to ensure we don't cut off text
    padding = 5
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(img.shape[1], x2 + padding)
    y2 = min(img.shape[0], y2 + padding)
    
    # Crop table region
    table_img = img[y1:y2, x1:x2]
    temp_path = Path(OUTPUT_DIR) / "temp_table.png"
    cv2.imwrite(str(temp_path), table_img)
    
    # Extract with Google Vision
    try:
        vision_blocks, vision_text = extract_text_google_vision(str(temp_path))
        
        result = {
            "text_blocks": vision_blocks,
            "full_text": vision_text
        }
    except Exception as e:
        print(f"    ⚠️  Warning: OCR failed - {str(e)}")
        result = {
            "text_blocks": [],
            "full_text": f"[OCR Error: {str(e)}]"
        }
    finally:
        # Clean up temp file
        if temp_path.exists():
            temp_path.unlink()
    
    return result


print("✅ Text extraction functions defined (COMPLETE)")

✅ Text extraction functions defined (COMPLETE)


In [50]:
# Cell 9: Main Pipeline Function (FIXED)
# ---------------- MAIN PIPELINE ----------------

def is_text_inside_table(text_block, table_boxes):
    """Check if a text block is inside any table region."""
    # Get text block coordinates
    if not text_block.get("bounding_box"):
        return False
    
    bbox = text_block["bounding_box"]
    if len(bbox) < 4:
        return False
    
    # Calculate text block center point
    text_x = sum([v.get("x", 0) for v in bbox]) / len(bbox)
    text_y = sum([v.get("y", 0) for v in bbox]) / len(bbox)
    
    # Check if center point is inside any table
    for table_box in table_boxes:
        x1, y1, x2, y2 = table_box
        if x1 <= text_x <= x2 and y1 <= text_y <= y2:
            return True
    
    return False

def run_pipeline(pdf_path):
    """Main processing pipeline."""
    print("\n" + "="*70)
    print("🚀 STABLE TABLE DETECTION & TEXT EXTRACTION")
    print("="*70 + "\n")
    
    pages = pdf_to_images(pdf_path)
    result = {"success": True, "method": "DETR + OpenCV + Google Vision", "pages": []}
    
    # For formatted text output (tables only)
    formatted_text_tables = []
    
    # For content text output (non-table text only with table location comments)
    content_text = []
    content_text.append("="*70)
    content_text.append("BPHS PDF - FULL TEXT CONTENT (Excluding Table Data)")
    content_text.append("="*70)
    content_text.append("")

    for page_num, page_img in enumerate(pages, start=1):
        print(f"\n{'='*70}")
        print(f"📄 Processing Page {page_num}/{len(pages)}")
        print(f"{'='*70}")

        img = cv2.imread(page_img)

        # DUAL-METHOD TABLE DETECTION
        table_boxes = detect_tables_all_methods(page_img)
        tables = []

        # Draw and extract text from each table
        for idx, box in enumerate(table_boxes, start=1):
            x1, y1, x2, y2 = map(int, box)
            
            # Draw detection box
            draw_dashed_rect(img, (x1, y1), (x2, y2))

            # Add label
            label = f"table_{idx}"
            label_pos = (x1 + 5, y1 - 10 if y1 > 30 else y1 + 25)
            
            (text_width, text_height), _ = cv2.getTextSize(
                label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2
            )
            cv2.rectangle(
                img,
                (label_pos[0] - 2, label_pos[1] - text_height - 2),
                (label_pos[0] + text_width + 2, label_pos[1] + 2),
                (0, 255, 0), -1
            )
            
            # FIX: Add all required parameters to putText
            cv2.putText(
                img, 
                label, 
                label_pos, 
                cv2.FONT_HERSHEY_SIMPLEX,  # font
                0.7,                        # fontScale
                (0, 0, 0),                  # color (black text)
                2,                          # thickness
                cv2.LINE_AA                 # lineType
            )

            # Extract text from table
            print(f"  📋 Extracting Table {idx}...")
            table_text = extract_table_text(page_img, box)
            
            tables.append({
                "table_id": f"table_{idx}",
                "bbox": box,
                "text": table_text
            })
            
            # Add to formatted output
            formatted_text_tables.append(f"\n{'='*70}")
            formatted_text_tables.append(f"Page {page_num} - Table {idx}")
            formatted_text_tables.append(f"{'='*70}")
            formatted_text_tables.append(table_text["full_text"])

        # Save annotated image
        output_img_path = Path(OUTPUT_DIR) / f"page_{page_num}_annotated.png"
        cv2.imwrite(str(output_img_path), img)
        print(f"  ✅ Saved annotated image: {output_img_path.name}")

        # Extract full page text for non-table content
        print(f"  📝 Extracting full page text...")
        page_blocks, page_text = extract_text_google_vision(page_img)
        
        # Add page header to content text
        content_text.append(f"\n{'='*70}")
        content_text.append(f"PAGE {page_num}")
        content_text.append(f"{'='*70}\n")
        
        # Filter out text that's inside tables
        non_table_blocks = []
        for block in page_blocks:
            if not is_text_inside_table(block, table_boxes):
                non_table_blocks.append(block)
        
        # Add non-table text
        for block in non_table_blocks:
            content_text.append(block.get("text", ""))
        
        # Add table location markers
        if table_boxes:
            content_text.append(f"\n[{len(table_boxes)} table(s) detected on this page - see tables.txt for content]")

        result["pages"].append({
            "page": page_num,
            "image": str(output_img_path),
            "table_count": len(tables),
            "tables": tables
        })

    # Save formatted table text
    table_text_path = Path(OUTPUT_DIR) / "tables.txt"
    with open(table_text_path, "w", encoding="utf-8") as f:
        f.write("\n".join(formatted_text_tables))
    print(f"\n✅ Saved table text: {table_text_path}")

    # Save non-table content text
    content_text_path = Path(OUTPUT_DIR) / "content.txt"
    with open(content_text_path, "w", encoding="utf-8") as f:
        f.write("\n".join(content_text))
    print(f"✅ Saved content text: {content_text_path}")

    # Save JSON
    json_path = Path(OUTPUT_DIR) / "results.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(result, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved JSON: {json_path}")

    return result

print("✅ Main pipeline function defined (FIXED)")

✅ Main pipeline function defined (FIXED)


In [51]:
#  Cell 10: Run the Pipeline
# ---------------- EXECUTE PROCESSING ----------------

if __name__ == "__main__":
    result = run_pipeline(PDF_PATH)
    
    # Display summary
    print("\n" + "="*70)
    print("📊 SUMMARY STATISTICS")
    print("="*70)
    
    for page_data in result["pages"]:
        page_num = page_data["page"]
        table_count = page_data["table_count"]
        print(f"  Page {page_num:2d}: {table_count} table(s) detected")
    
    print("\n✅ All files saved in:", OUTPUT_DIR)
    print("="*70)


🚀 STABLE TABLE DETECTION & TEXT EXTRACTION

Converting PDF to images (DPI: 300)...
✅ Converted 25 pages

📄 Processing Page 1/25
  🔍 DETR Pass 1 (threshold 0.6)...
    Found: 1 tables
  🔍 DETR Pass 2 (threshold 0.4)...
    Found: 1 tables
  🔍 OpenCV contour detection...
    Found: 0 tables
  📦 Total boxes before merging: 2
  ✅ Final unique tables: 1
  📋 Extracting Table 1...
  ✅ Saved annotated image: page_1_annotated.png
  📝 Extracting full page text...

📄 Processing Page 2/25
  🔍 DETR Pass 1 (threshold 0.6)...
    Found: 0 tables
  🔍 DETR Pass 2 (threshold 0.4)...
    Found: 2 tables
  🔍 OpenCV contour detection...
    Found: 2 tables
  📦 Total boxes before merging: 4
  ✅ Final unique tables: 2
  📋 Extracting Table 1...
  📋 Extracting Table 2...
  ✅ Saved annotated image: page_2_annotated.png
  📝 Extracting full page text...

📄 Processing Page 3/25
  🔍 DETR Pass 1 (threshold 0.6)...
    Found: 1 tables
  🔍 DETR Pass 2 (threshold 0.4)...
    Found: 1 tables
  🔍 OpenCV contour detectio